In [14]:
import pandas as pd
import numpy as np 


#import two csv's to DataFrames
pl = pd.read_csv('Mar bud v act.csv').fillna(0)
pl_map = pd.read_csv('P&L Mapping File.csv').fillna(0)

#trim pl Account Column
pl['Account'] = pl['Account'].str.strip()

#join DataFrames
merged = pl.merge(pl_map, left_on='Account', right_on ="Account")

#set csv index to 'Account Column'
merged_indexed = merged.set_index('Account',drop=False)



#Remove $ from 'Total' in df
merged_indexed['Actual'] = merged_indexed['Actual'].str.replace(r'$', '',regex=True)
merged_indexed['Budget'] = merged_indexed['Budget'].str.replace(r'$', '',regex=True)


#remove commas from dataFrame
merged_indexed['Actual'] = (merged_indexed['Actual'].str.split()).apply(lambda x: float(x[0].replace(',', '')))
merged_indexed['Budget'] = (merged_indexed['Budget'].str.split()).apply(lambda x: float(x[0].replace(',', '')))



#create dataFrame that pulls in specifc rows and columns
unordered_pl = merged_indexed.loc[["Total Amazon Sales", "Total Sales from Website",'Total Income',"Total Cost of Goods Sold", "Gross Profit", "Total Legal/Professional Fees",'Total Salaries & Benefits',"Total Analogue Ads","Total Digital Ads","Retail Marketing","Total Marketing Expense","Total Research & Development","Net Other Income","Net Income"],['Account','Actual','Budget']]



#sum specific rows and columns

unordered_pl.loc['Retail and Wholesale','Actual'] = unordered_pl.loc[['Total Income'],'Actual'].sum() - unordered_pl.loc[['Total Amazon Sales','Total Sales from Website'],'Actual'].sum()
unordered_pl.loc['Retail and Wholesale','Budget'] = unordered_pl.loc[['Total Income'],'Budget'].sum() - unordered_pl.loc[['Total Amazon Sales','Total Sales from Website'],'Budget'].sum()
unordered_pl.loc['Other Expenses','Actual'] = unordered_pl.loc[['Gross Profit'],'Actual'].sum() + unordered_pl.loc[['Net Other Income'],'Actual'].sum() - unordered_pl.loc[['Net Income'],'Actual'].sum() - unordered_pl.loc[[ "Total Legal/Professional Fees",'Total Salaries & Benefits',"Total Marketing Expense","Total Research & Development"],'Actual'].sum()
unordered_pl.loc['Other Expenses','Budget'] = unordered_pl.loc[['Gross Profit'],'Budget'].sum() + unordered_pl.loc[['Net Other Income'],'Budget'].sum() - unordered_pl.loc[['Net Income'],'Budget'].sum() - unordered_pl.loc[[ "Total Legal/Professional Fees",'Total Salaries & Benefits',"Total Marketing Expense","Total Research & Development"],'Budget'].sum()
unordered_pl.loc['Other Marketing','Actual'] = unordered_pl.loc[['Total Marketing Expense'],'Actual'].sum() - unordered_pl.loc[[ "Total Analogue Ads","Total Digital Ads","Retail Marketing"],'Actual'].sum()
unordered_pl.loc['Other Marketing','Budget'] = unordered_pl.loc[['Total Marketing Expense'],'Budget'].sum() - unordered_pl.loc[[ "Total Analogue Ads","Total Digital Ads","Retail Marketing"],'Budget'].sum()



#Add variance column
unordered_pl = unordered_pl.assign(variance=np.where(unordered_pl['Account'].isin(["Total Amazon Sales","Total Income","Total Income","Gross Profit", "Net Other Income","Net Income"]),unordered_pl['Actual'] - unordered_pl["Budget"] ,unordered_pl["Budget"] - unordered_pl['Actual'] ))


#Add % of budget column
unordered_pl['% of Budget'] = unordered_pl['Actual'] / unordered_pl['Budget']



#reorder rows to proper P&L Format
ordered_pl = unordered_pl.reindex(["Total Amazon Sales", "Total Sales from Website", "Retail and Wholesale",'Total Income',"Total Cost of Goods Sold", "Gross Profit", "Total Legal/Professional Fees",'Total Salaries & Benefits',"Total Analogue Ads","Total Digital Ads","Retail Marketing","Other Marketing","Total Marketing Expense","Total Research & Development","Other Expenses","Net Other Income","Net Income"])

#Replace 'Account' Column with index so every row shoes up
ordered_pl['Account'] = ordered_pl.index





ordered_pl.to_csv('/Users/evandiazdearce/Documents/output/pl_adjusted.csv', index = False, header=True)
